# Latin Line Break (`<lb/>`) Detector

This notebook demonstrates how to use the Flair model `mschonhardt/latin-contextual-lb-detector`.
It detects if a line break `<lb/>` in Latin text acts as a 
**word break** (hyphenation) or a **separate word** (space).

## Setup Environment

In [ ]:
# Install flair if running in Colab or if not installed locally
# !pip install -q flair
# Flair 0.15.1 recommended

import flair
print(f"Flair version: {flair.__version__}")

print("Environment ready.")

Flair version: 0.15.1
Environment ready.


## Load the Model from Hugging Face

In [10]:
from flair.models import SequenceTagger
from flair.data import Sentence

print("Loading model: mschonhardt/latin-contextual-lb-detector ...")
tagger = SequenceTagger.load('mschonhardt/latin-contextual-lb-detector')
print("Model loaded successfully!")

Loading model: mschonhardt/latin-contextual-lb-detector ...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:  96%|#########5| 503M/526M [00:00<?, ?B/s]

c:\Users\Michael Schonhardt\Documents\Github\notebooks\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Michael Schonhardt\.flair\models\latin-contextual-lb-detector\models--mschonhardt--latin-contextual-lb-detector. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


2026-02-11 20:45:45,899 SequenceTagger predicts: Dictionary with 5 tags: O, NB, WB, <START>, <STOP>
Model loaded successfully!


### Prediction Logic
We must tokenize by **whitespace** so that `<lb/>` is treated as a single token.

In [11]:
def predict_line_breaks(text_input):
    # Important: Model expects '<lb/>' as token. Tokenize by whitespace to ensure <lb/> is 
    # preserved as one token and not split into '<', 'lb/', '>'
    # Input: "line1 <lb/> line2" -> ["line1", "<lb/>", "line2"]
    token_list = text_input.split()
    if not token_list: return []

    # Create Flair Sentence
    sentence = Sentence(token_list)

    # Predict
    tagger.predict(sentence)

    # Extract Results
    results = []
    for token in sentence:
        if "<lb" in token.text:
            tag = token.get_label().value
            score = token.get_label().score
            
            # Map tags to human-readable meanings
            meaning = "Separate Words (Space)" if tag == "NB" else "Split Word (Join)"
            
            results.append({
                "token": token.text,
                "prediction": tag,
                "confidence": score,
                "meaning": meaning
            })
            
    return results

### Run Inference

In [12]:
# Example 1: A split words of lines
text1 = "imper <lb/> ator"
print(f"Input: {text1}")
print(predict_line_breaks(text1))

print("-" * 30)

# Example 2: Separate words of lines
text2 = "et in terra <lb/> pax hominibus"
print(f"Input: {text2}")
print(predict_line_breaks(text2))

Input: imper <lb/> ator
[{'token': '<lb/>', 'prediction': 'WB', 'confidence': 0.9857004880905151, 'meaning': 'Split Word (Join)'}]
------------------------------
Input: et in terra <lb/> pax hominibus
[{'token': '<lb/>', 'prediction': 'NB', 'confidence': 0.99724280834198, 'meaning': 'Separate Words (Space)'}]
